# Import Functions

In [11]:
from small_world_functions import find_best_bridges, find_best_bridges_from_ydk, ydk_to_df_adjacency_matrix

# Find Best Bridges From List of Names

In [12]:
#cards that we want to find bridges between
deck_monster_names = ['Ash Blossom & Joyous Spring',
                        'Effect Veiler',
                        'Ghost Belle & Haunted Mansion',
                        'Mathmech Addition',
                        'Mathmech Circular',
                        'Mathmech Diameter',
                        'Mathmech Multiplication',
                        'Mathmech Nabla',
                        'Mathmech Sigma',
                        'Mathmech Subtraction',
                        'Maxx "C"']

#cards that are required to connect with a bridge
required_target_names = ['Mathmech Circular',
                        'Maxx "C"',]

bridges = find_best_bridges(deck_monster_names, required_target_names)
bridges.head(20)

,number_of_connections,name,type,attribute,level,atk,def
0,10.0,Hiita the Fire Charmer,Spellcaster,FIRE,3.0,500.0,1500.0
1,10.0,Numbing Grub in the Ice Barrier,Insect,WATER,4.0,1300.0,1800.0
2,9.0,Amorphage Envy,Dragon,EARTH,4.0,0.0,2050.0
3,9.0,Amorphage Greed,Dragon,EARTH,4.0,0.0,1950.0
4,9.0,Amorphage Pride,Dragon,EARTH,4.0,1750.0,0.0
5,9.0,Amorphage Wrath,Dragon,EARTH,4.0,1650.0,0.0
6,9.0,Anarchist Monk Ranshin,Spellcaster,EARTH,4.0,1800.0,100.0
7,9.0,Angraecum Umbrella,Spellcaster,WATER,3.0,500.0,1500.0
8,9.0,Beckoned by the World Chalice,Warrior,EARTH,4.0,1800.0,0.0
9,9.0,Boot-Up Corporal - Command Dynamo,Machine,EARTH,4.0,0.0,2000.0


# Find Best Bridges From YDK File

In [13]:
ydk_file = 'mathmech_deck.ydk'
bridges_from_ydk = find_best_bridges_from_ydk(ydk_file) #find the best Small World bridges for your deck
bridges_from_ydk.head(20)

,number_of_connections,name,type,attribute,level,atk,def
0,11.0,Backlinker,Cyberse,DARK,3.0,1600.0,0.0
1,11.0,Defcon Bird,Cyberse,WIND,3.0,100.0,1700.0
2,11.0,Guard Penguin,Winged Beast,WATER,4.0,0.0,1200.0
3,11.0,Super Crashbug,Fiend,DARK,4.0,0.0,3000.0
4,10.0,Adamancipator Crystal - Dragite,Rock,WATER,4.0,0.0,2200.0
5,10.0,Adamancipator Crystal - Raptite,Rock,WIND,4.0,0.0,2200.0
6,10.0,Alchemist of Black Spells,Spellcaster,WIND,4.0,1200.0,1800.0
7,10.0,Blue Mountain Butterspy,Warrior,DARK,4.0,0.0,1700.0
8,10.0,Capricious Darklord,Fairy,DARK,4.0,0.0,1600.0
9,10.0,D/D Ark,Fiend,DARK,4.0,0.0,2000.0


# Small World Adjacency Matrix

In [14]:
mathmech_adjacency_matrix = ydk_to_df_adjacency_matrix(ydk_file)
mathmech_adjacency_matrix #if an entry corresponding to two cards is non-zero, then one card connects to the other with Small World

,Archfiend Eccentrick,Ash Blossom & Joyous Spring,D.D. Crow,Effect Veiler,Ghost Belle & Haunted Mansion,Mathmech Addition,Mathmech Circular,Mathmech Diameter,Mathmech Multiplication,Mathmech Nabla,Mathmech Sigma,Mathmech Subtraction
Archfiend Eccentrick,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0
Ash Blossom & Joyous Spring,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
D.D. Crow,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
Effect Veiler,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
Ghost Belle & Haunted Mansion,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
Mathmech Addition,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Mathmech Circular,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Mathmech Diameter,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Mathmech Multiplication,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Mathmech Nabla,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Small World Adjacency Matrix Squared

In [15]:
mathmech_adjacency_matrix_squared = ydk_to_df_adjacency_matrix(ydk_file, squared=True)
mathmech_adjacency_matrix_squared #if an entry corresponding to two cards is non-zero, then one card is searchable from the other with Small World

,Archfiend Eccentrick,Ash Blossom & Joyous Spring,D.D. Crow,Effect Veiler,Ghost Belle & Haunted Mansion,Mathmech Addition,Mathmech Circular,Mathmech Diameter,Mathmech Multiplication,Mathmech Nabla,Mathmech Sigma,Mathmech Subtraction
Archfiend Eccentrick,8.0,3.0,1.0,5.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0
Ash Blossom & Joyous Spring,3.0,4.0,1.0,1.0,2.0,1.0,2.0,2.0,0.0,0.0,2.0,1.0
D.D. Crow,1.0,1.0,2.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
Effect Veiler,5.0,1.0,0.0,7.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0
Ghost Belle & Haunted Mansion,1.0,2.0,1.0,1.0,3.0,1.0,2.0,2.0,0.0,0.0,2.0,1.0
Mathmech Addition,1.0,1.0,0.0,2.0,1.0,2.0,1.0,1.0,0.0,0.0,1.0,2.0
Mathmech Circular,1.0,2.0,1.0,1.0,2.0,1.0,2.0,2.0,0.0,0.0,2.0,1.0
Mathmech Diameter,1.0,2.0,1.0,1.0,2.0,1.0,2.0,2.0,0.0,0.0,2.0,1.0
Mathmech Multiplication,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
Mathmech Nabla,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
